*langchain*

In [1]:
!pip install requests-cache python-dotenv 


  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached cattrs-23.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl.metadata (3.1 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests_cache-1.2.1-py3-none-any.whl (61 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached cattrs-23.2.3-py3-none-any.whl (57 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)


In [2]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [
    "Silo Pharma",
    "optimi health",
    "Tilray",
    "Numinus Wellness",
    "MindMed",
    "Compass Pathways",
    "Field Trip Health",
    "Cybin",
    "ATAI Life Sciences",
    "Seelos Therapeutics",
    "Mind Cure Health",
    "Tryp Therapeutics",
    "Revive Therapeutics",
    "Red Light Holland",
    "Hollister Biosciences",
    "Mydecine Innovations Group",
    "Numinus Wellness",
    "cronosgroup",
    "Ascend Wellness Holdings",
    "Verano Holdings",
    "Columbia Care",
    "Green Thumb Industries",
    "Curaleaf Holdings",
    "Trulieve Cannabis",
    "SNDL Inc",
    "IM Cannabis",
    "4front ventures",
    "Ayr Wellness",
    "TerrAscend",
    "Cresco Labs",
    "Harvest Health & Recreation",
    "Jushi Holdings",
    "MedMen",
    "Planet 13 Holdings",
    "Vireo Health",
    "Cannabis One",
    "Cansortium",
    "Columbia Care",
    "AFC Gamma"  
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=20)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# No guardar en archivo JSON, continuar con el procesamiento en memoria

Buscando noticias para: Silo Pharma
Encontradas 3 noticias para Silo Pharma
Buscando noticias para: optimi health
Encontradas 2 noticias para optimi health
Buscando noticias para: Tilray
Encontradas 51 noticias para Tilray
Buscando noticias para: Numinus Wellness
Encontradas 2 noticias para Numinus Wellness
Buscando noticias para: MindMed
Encontradas 18 noticias para MindMed
Buscando noticias para: Compass Pathways
Encontradas 41 noticias para Compass Pathways
Buscando noticias para: Field Trip Health
Encontradas 100 noticias para Field Trip Health
Buscando noticias para: Cybin
Encontradas 8 noticias para Cybin
Buscando noticias para: ATAI Life Sciences
Encontradas 8 noticias para ATAI Life Sciences
Buscando noticias para: Seelos Therapeutics
Encontradas 4 noticias para Seelos Therapeutics
Buscando noticias para: Mind Cure Health
Encontradas 65 noticias para Mind Cure Health
Buscando noticias para: Tryp Therapeutics
Encontradas 1 noticias para Tryp Therapeutics
Buscando noticias para: 

In [11]:
!pip install --upgrade langchain chromadb langchain-text-splitters openai tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 7.5 MB/s eta 0:00:00


In [9]:
!pip install -U langchain-community

In [12]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for company, articles in news_data.items():
    if isinstance(articles, list):
        for article in articles:
            title = article.get('title', 'No title')
            snippet = article.get('snippet', 'No snippet')
            link = article.get('link', 'No link')
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title'or'snippet:Nosi' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

/home/ozz/miniconda3/envs/llms_lab/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre la empresa?",
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre la empresa",
    "Podrias compartirme los links de las noticas"
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre la empresa?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre la empresa?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre la empresa?",
]


# Almacenar las respuestas en una lista
responses = []

for empresa in news_data:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")



Analizando preguntas para: Silo Pharma


/home/ozz/miniconda3/envs/llms_lab/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Pregunta 1/5 analizada para Silo Pharma
Pregunta 2/5 analizada para Silo Pharma
Pregunta 3/5 analizada para Silo Pharma
Pregunta 4/5 analizada para Silo Pharma
Pregunta 5/5 analizada para Silo Pharma
Analizando preguntas para: optimi health
Pregunta 1/5 analizada para optimi health
Pregunta 2/5 analizada para optimi health
Pregunta 3/5 analizada para optimi health
Pregunta 4/5 analizada para optimi health
Pregunta 5/5 analizada para optimi health
Analizando preguntas para: Tilray
Pregunta 1/5 analizada para Tilray
Pregunta 2/5 analizada para Tilray
Pregunta 3/5 analizada para Tilray
Pregunta 4/5 analizada para Tilray
Pregunta 5/5 analizada para Tilray
Analizando preguntas para: Numinus Wellness
Pregunta 1/5 analizada para Numinus Wellness
Pregunta 2/5 analizada para Numinus Wellness
Pregunta 3/5 analizada para Numinus Wellness
Pregunta 4/5 analizada para Numinus Wellness
Pregunta 5/5 analizada para Numinus Wellness
Analizando preguntas para: MindMed
Pregunta 1/5 analizada para MindMed


In [15]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses)

[         {         'empresa': 'Silo Pharma',
                    'query': '¿Cuáles son los temas más mencionados en las '
                             'noticias recientes sobre la empresa?',
                    'response': 'Los temas más mencionados en las noticias '
                                'recientes sobre Silo Pharma incluyen el '
                                'desarrollo de un implante terapéutico de '
                                'ketamina para el tratamiento del dolor '
                                'crónico y la fibromialgia, así como una '
                                'asociación para avanzar en el desarrollo de '
                                'un péptido que se dirige al sistema nervioso '
                                'central, con un enfoque inicial en la '
                                'esclerosis múltiple.'},
          {         'empresa': 'Silo Pharma',
                    'query': '¿Qué titulares de noticias se repiten entre los '
                

<h1> Cannabis en general <h1>


In [16]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de temas relacionados con cannabis
temas = [
    "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psilocybin"
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 20 días)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = {}

for tema in temas:
    print(f"Buscando noticias para: {tema}")
    #json para buscar temas en la API
    params = {
        'q': tema,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }
    #Aqui hago la requests a la API
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        if articles:
            news_data[tema] = []
            for article in articles:
                title = article['title']
                link = article['url']
                snippet = article['description']
                published_at = article['publishedAt']
                news_data[tema].append(
                    {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
            print(f"Encontradas {len(articles)} noticias para {tema}")
        else:
            print(f"No se encontraron noticias para {tema}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {tema}: {str(e)}")

# Filtrar temas sin noticias
news_data = {tema: articles for tema, articles in news_data.items() if articles}

# No guardar en archivo JSON, continuar con el procesamiento en memoria

Buscando noticias para: Cannabis legalization
Encontradas 32 noticias para Cannabis legalization
Buscando noticias para: Medical marijuana
Encontradas 51 noticias para Medical marijuana
Buscando noticias para: Cannabis industry
Encontradas 70 noticias para Cannabis industry
Buscando noticias para: Cannabis investment
Encontradas 54 noticias para Cannabis investment
Buscando noticias para: Cannabis stocks
Encontradas 12 noticias para Cannabis stocks
Buscando noticias para: Cannabis research
Encontradas 100 noticias para Cannabis research
Buscando noticias para: Cannabis news
Encontradas 100 noticias para Cannabis news
Buscando noticias para: Cannabis policy
Encontradas 31 noticias para Cannabis policy
Buscando noticias para: Cannabis business
Encontradas 100 noticias para Cannabis business
Buscando noticias para: Cannabis culture
Encontradas 10 noticias para Cannabis culture
Buscando noticias para: PsilocybinPsychedelics research
No se encontraron noticias para PsilocybinPsychedelics re

In [17]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for tema, articles in news_data.items():
    for article in articles:
        title = article.get('title', 'No title')
        snippet = article.get('snippet', 'No snippet')
        link = article.get('link', 'No link')
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in extracted_elements]

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)

#instancia de la base de datos
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [18]:
questions = [
    "Cuales son los temas mas populares y mencionados en las noticias recientes sobre el tema, menciona los temas",
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre el tema?",
    "Podrias traerme los links de esas noticias?"
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre el tema?",
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre el tema?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre el tema?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre el tema?"
]

# Almacenar las respuestas en una lista
responses = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses.append({
            "tema": tema,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")



Analizando preguntas para: Cannabis legalization
Pregunta 1/6 analizada para Cannabis legalization
Pregunta 2/6 analizada para Cannabis legalization
Pregunta 3/6 analizada para Cannabis legalization
Pregunta 4/6 analizada para Cannabis legalization
Pregunta 5/6 analizada para Cannabis legalization
Pregunta 6/6 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/6 analizada para Medical marijuana
Pregunta 2/6 analizada para Medical marijuana
Pregunta 3/6 analizada para Medical marijuana
Pregunta 4/6 analizada para Medical marijuana
Pregunta 5/6 analizada para Medical marijuana
Pregunta 6/6 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/6 analizada para Cannabis industry
Pregunta 2/6 analizada para Cannabis industry
Pregunta 3/6 analizada para Cannabis industry
Pregunta 4/6 analizada para Cannabis industry
Pregunta 5/6 analizada para Cannabis industry
Pregunta 6/6 analizada para Cannabis industry
Analizand

In [19]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

[       {       'query': 'Cuales son los temas mas populares y mencionados en '
                         'las noticias recientes sobre el tema, menciona los '
                         'temas',
                'response': 'Los temas más populares y mencionados en las '
                            'noticias recientes sobre la legalización del '
                            'cannabis incluyen:\n'
                            '\n'
                            '1. **Opiniones de los votantes**: Las encuestas '
                            'muestran que los votantes en estados clave están '
                            'abrumadoramente a favor de la legalización y '
                            'regulación del cannabis para uso adulto.\n'
                            '\n'
                            '2. **Impacto político**: Se destaca que ignorar '
                            'las opiniones de los electores sobre la '
                            'legalización del cannabis puede tener '
           